In [8]:
import pandas as pd
from datetime import date, timedelta
from tqdm import tqdm
from sodapy import Socrata

In [9]:
sdate = date(2020, 3, 28)   # start date
# sdate = date(2020, 7, 1)   # start date

edate = date.today()   # end date
delta = edate - sdate       # as timedelta

dates = []
for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    dates.append(str(day))

In [10]:
dicc_names = {
    "Caso": ["No. Caso", "ID de caso", "CASO", "id_de_caso"],
    "Tipo": ["Tipo*", "fuente o tipo de contagio", "Fuente_tipo_contagio"],
    "Pais de procedencia" : ["País de procedencia", "Pais de procedencia\t", "Pais a donde viajo  1"],
    "Ciudad": ["Ciudad de ubicación", "Ciudad/Municipio", "Ciudad_municipio", "ciudad_de_ubicaci_n"],
    "Código País": ["Codigo pais"],
    "Ubicación": ["Atención**", "Ubicacion", "atenci_n"],
    "Departamento": ["Departamento o Distrito", "nombre_depa", "departamento"],
    "Fecha_diagnostico": ["Fecha diagnostico", "Fechadiagnostico", "Fecha de diagnóstico", "Fecha de diagnostico"],
    "Fecha_inicio_sintomas": ["Fechadeiniciodesíntomas", "Fecha de inicio de síntomas", "fis"],
    "Tipo_recuperacion": ["Tipo de recuperación", "Tipo_Recuperacion"],
    "Fecha_recuperado": ["Fecha de recuperado", "Fecha recuperado"],
    "Recuperacion": ["Tipo_recuperacion"],
    "Edad": ["edad"],
    "Sexo": ["sexo"]
}
cols2drop = [
    'Unnamed: 1',
    'Pais_viajo_1_cod',
    'Pais_viajo_1_nom',
    'Estado.1',
    "Pais_viajo_1"
]

In [11]:
def rename_cols(df):
    if "Departamento" in df.columns:
        if df["Departamento"].dtype.str == "<i8":
            df = df.drop("Departamento", axis = 1)
    for column in df.columns:
        for name in dicc_names:
            if column in dicc_names[name]:
                df = df.rename(columns={column:name})
    return df

In [12]:
def load_colombia_df():
    client = Socrata("www.datos.gov.co", None)  # https://www.datos.gov.co/es/profile/edit/developer_settings   por si no funciona
    results = client.get("gt2j-8ykr", limit=1000000)
    results_df = pd.DataFrame.from_records(results)
    return results_df

In [18]:
big_df = pd.DataFrame()

for date in tqdm(dates):
    try:
        link =f'https://www.ins.gov.co/BoletinesCasosCOVID19Colombia/{date}.xlsx'
        data = pd.read_excel(link)
    except Exception as e:
        print("ended in " + date)
        print(f"error: {e}")
        continue
        
    data["fecha_boletin"] = date
    data_aux = data.copy(deep=True)
    data = rename_cols(data)
    data = data.drop(cols2drop, axis=1, errors="ignore")
    data["Departamento"] = data["Departamento"].str.lower() 
    data = data[data["Departamento"] == "antioquia"]
    big_df = big_df.append(data)


# big_df = big_df.loc[:,["fecha_boletin", "Caso", "Departamento", "Ciudad", "Edad", "Sexo", "Ubicación", "Estado", "Código ciudad"]]

 10%|█         | 11/106 [00:09<01:18,  1.20it/s]


KeyboardInterrupt: 

In [16]:
big_df["fecha_boletin"]

2         2020-03-28
3         2020-03-28
4         2020-03-28
5         2020-03-28
19        2020-03-28
             ...    
137742    2020-07-11
137751    2020-07-11
137778    2020-07-11
137779    2020-07-11
137823    2020-07-11
Name: fecha_boletin, Length: 154936, dtype: object

In [17]:
big_df.to_csv("julio11.csv", index=False)